# Table of contents:

* [Introduction to block ciphers](#intro-block)
* [Padding a message](#message-padding)
* [The Advanced Encryption Standard (AES)](#AES)
* [Modes of operation of block ciphers](#modes)
* [Size of the output ciphertex on AES](#size)
* [Bonus: Fernet cipher](#fernet)
    
Author: [Sebastià Agramunt Puig](https://github.com/sebastiaagramunt) for [OpenMined](https://www.openmined.org/) Privacy ML Series course.



## Block Ciphers <a class="anchor" id="intro-block"></a>

Block ciphers as opposed to stream ciphers take a block of the plaintext (a specific amount of bytes) and encrypts it into a block with the same size. In this section we will use the Advanced Encryption Standard (AES) to understand block ciphers. In the next schema it is shown how an original message of arbitrary $N$ bytes is converted into a ciphertext having blocks of $K$ bytes. The ciphertext size is always a multiple of $K$ bytes.

In [4]:
%pip install pyaes

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for pyaes: started
  Running setup.py install for pyaes: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: pyaes is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install pbkdf2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pbkdf2: filename=pbkdf2-1.3-py3-none-any.whl size=5103 sha256=93b2129688a23edeb8239fc949beff5a32f32891d819bbda507d8eee225abca2
  Stored in directory: /root/.cache/pip/wheels/49/16/ea/daca297d70ee0782ac6e16e83b2c55b2ca42a2113750bc0489
Successfully built pbkdf2


In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

data = b'secret data'

key = get_random_bytes(16)
cipher = AES.new(key, AES.MODE_EAX)
ciphertext, tag = cipher.encrypt_and_digest(data)
nonce = cipher.nonce

ModuleNotFoundError: ignored

<img src="img/block_cipher.png" style="width:1100px"/>

## Padding a message <a class="anchor" id="message-padding"></a>

Most of the times the lenght of the message is not a multiple of the block size so we need to "pad" the message to have the required length. A common padding function is [PKCS7](https://en.wikipedia.org/wiki/Padding_(cryptography)). Basically what PKCS7 does is appendinng a list of bytes with the same value corresponding to the number of bytes needed to complete the block.




## Encrypting using AES (Advanced Encryption Standard) <a class="anchor" id="AES"></a>

AES is a block cipher that was established as a standard by NIST in 2001 (after a public call to improve/substitute DES encryption algorithm in 1997). AES is a subset of the Rijndael block cipher developed by Vincent Rijmen and Joan Daemen submitted to NIST during the [AES selection process](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard_process).


We are not going to go into the details of te exact implementation but the readers are referred to the book of [Katz and Lindell](http://www.cs.umd.edu/~jkatz/imc.html) Chapter 6 section 2. Also Mike Pound explains AES in this [video](https://www.youtube.com/watch?v=O4xNJsjtN6E&t=524s&ab_channel=Computerphile), check it out!

In [ ]:
!pip install matplotlib-venn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get -qq install -y libfluidsynth1

In [1]:
%pip install tfp-nightly

     ---------------------------------------- 6.7/6.7 MB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 90.9/90.9 kB 5.0 MB/s eta 0:00:00
     -------------------------------------- 124.6/124.6 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!apt-get install libmagic-dev
!pip install python-magic


SyntaxError: invalid syntax (1453081399.py, line 3)

In [ ]:
import magic


In [ ]:
from Crypto.Cipher import AES
from Crypto import Random

ModuleNotFoundError: ignored

In [ ]:
from Crypto.Cipher import AES
    from Crypto import Random

    BLOCK_SIZE = 32

    INTERRUPT = u'\u0001'

    PAD = u'\u0000'

    def AddPadding(data, interrupt, pad, block_size):
        new_data = ''.join([data, interrupt])
        new_data_len = len(new_data)
        remaining_len = block_size - new_data_len
        to_pad_len = remaining_len % block_size
        pad_string = pad * to_pad_len
        return ''.join([new_data, pad_string])

    def StripPadding(data, interrupt, pad):
        return data.rstrip(pad).rstrip(interrupt)

    SECRET_KEY = Random.new().read(32)

    IV = Random.new().read(16)


IndentationError: ignored

In [ ]:
!install pip
!pip install utils
import utils

install: missing destination file operand after 'pip'
Try 'install --help' for more information.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from utils import padding

ImportError: ignored

In [ ]:
pip uninstall pycrypto
easy_install pycrypto

SyntaxError: ignored

In [ ]:
import Crypto.Util package

SyntaxError: ignored

In [ ]:
from Crypto.Util.Padding import pad, unpad
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

ModuleNotFoundError: ignored

In [ ]:
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive
#from ..Util import util
from ..Util import padding

MODE_ECB = 1
MODE_CBC = 2


class BlockCipher():
    """ Base class for all blockciphers
    """

    key_error_message = "Wrong key size" #should be overwritten in child classes

    def __init__(self,key,mode,IV,counter,cipher_module,segment_size,args={}):
        # Cipher classes inheriting from this one take care of:
        #   self.blocksize
        #   self.cipher
        self.key = key
        self.mode = mode
        self.cache = ''
        self.ed = None

        if 'keylen_valid' in dir(self): #wrappers for pycrypto functions don't have this function
         if not self.keylen_valid(key) and type(key) is not tuple:
                raise ValueError(self.key_error_message)

        if IV == None:
            self.IV = '\x00'*self.blocksize
        else:
            self.IV = IV



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ImportError: ignored

## Modes of operation of block ciphers <a class="anchor" id="mode"></a>

A block cipher by itself is only suitable for the secure cryptographic transformation (encryption or decryption) of one fixed-length group of bits called a block. A mode of operation describes how to repeatedly apply a cipher's single-block operation to securely transform amounts of data larger than a block ([Wikipedia](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation)).

The first mode is "not doing anything", this is the Electronic Codebook mode. See the figure below (from Wikipedia).

<img src="img/ECB_mode.png" style="width:1100px"/>

\We are lucky and in ```cryptography``` package ECB implemented in ```cryptography.hazmat.primitives.ciphers.ECB``` function (we've seen in the previous example!).

:Now we can encrypt the same message twice and see what we get in the ciphertext:

This is not a desirable outcome. If I want to send the same message twice, I 

---

really don't want to send the same ciphertext. What if in all comunications I start by "Dear..." and the attacker knows it?. A better mode is the Cipher block chaining (CBC):

### <img src="img/CBC_mode.png" style="width:1100px"/>

In this case we take a random initialization vector and perform XOR operation with the block of plaintext, then we feed this into the encryptor, after that we obtain the ciphertext. This ciphertext is used as the initialization vector to encrypt the next block.

## Size of ciphertext <a class="anchor" id="size"></a>

## Bonus: Fernet <a class="anchor" id="fernet"></a>

Another block cipher implemented in cryptography package is [Fernet](https://asecuritysite.com/encryption/fernet). 